# Week 9 — Dimensionality Reduction: PCA

**Course:** Applied ML Foundations for SaaS Analytics  
**Week Focus:** Reduce high-dimensional data to 2-3 dimensions for visualization and denoising.

---

## 🎯 Learning Objectives

- Understand Principal Component Analysis (PCA)
- Reduce features while preserving variance
- Interpret principal components
- Use PCA for visualization and denoising
- Balance information retention vs simplification

In [ ]:
import pandas as pd, numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Load data
subs = pd.read_csv('../data/subscriptions.csv')
feature_usage = pd.read_csv('../data/feature_usage.csv')
user_events = pd.read_csv('../data/user_events.csv')

# Feature engineering
engagement = feature_usage.groupby('user_id').agg({'usage_count': 'sum', 'feature_name': 'nunique'}).reset_index()
engagement.columns = ['user_id', 'total_usage', 'features_adopted']

events = user_events.groupby('user_id').size().reset_index(name='total_events')

df = subs[['user_id', 'tenure_days', 'mrr']].merge(engagement, on='user_id', how='left')
df = df.merge(events, on='user_id', how='left').fillna(0)

# Derived features
df['usage_per_day'] = df['total_usage'] / (df['tenure_days'] + 1)
df['events_per_day'] = df['total_events'] / (df['tenure_days'] + 1)

features = ['tenure_days', 'mrr', 'total_usage', 'features_adopted', 'total_events', 'usage_per_day', 'events_per_day']
print(f"Original features: {len(features)}")

## Part 1: Curse of Dimensionality

**Problem:** More features ≠ better predictions
- Data becomes sparse
- Computational complexity explodes
- Noise becomes prominent
- Can't visualize >3D

**💡 Depth Note:** Test on high-dimensional datasets (200+ features). When does performance plateau or decrease?

In [ ]:
# Standardize
X = df[features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Data shape: {X_scaled.shape}")
print(f"Features: {X_scaled.shape[1]} dimensions")

## Part 2: PCA - Find Important Dimensions

In [ ]:
# Fit PCA on full data
pca_full = PCA()
pca_full.fit(X_scaled)

cumsum_var = np.cumsum(pca_full.explained_variance_ratio_)

print("=" * 60)
print("EXPLAINED VARIANCE BY COMPONENT")
print("=" * 60)

for i in range(len(pca_full.explained_variance_ratio_)):
    var_pct = 100 * pca_full.explained_variance_ratio_[i]
    cumsum_pct = 100 * cumsum_var[i]
    print(f"PC{i+1}: {var_pct:>5.1f}% | Cumulative: {cumsum_pct:>6.1f}%")

# Components needed for 95% variance
n_comp_95 = np.argmax(cumsum_var >= 0.95) + 1
print(f"\n✅ Components for 95% variance: {n_comp_95}")
print(f"✅ Dimensionality reduction: {len(features)} → {n_comp_95} ({100*n_comp_95/len(features):.0f}%)")

## Part 3: Component Interpretation

In [ ]:
# Reduce to 2D
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

var_2d = 100 * sum(pca_2d.explained_variance_ratio_)
print(f"2D PCA: {var_2d:.1f}% variance")

# PC1 interpretation
print("\nPC1 loadings (feature contributions):")
pc1_df = pd.DataFrame(
    {'Feature': features, 'Loading': pca_2d.components_[0]}
).sort_values('Loading', ascending=False)
print(pc1_df.to_string(index=False))

# PC2 interpretation
print("\nPC2 loadings (feature contributions):")
pc2_df = pd.DataFrame(
    {'Feature': features, 'Loading': pca_2d.components_[1]}
).sort_values('Loading', ascending=False)
print(pc2_df.to_string(index=False))

print("\n💡 Business Interpretation:")
print(f"   PC1: {pc1_df.iloc[0]['Feature']} + {pc1_df.iloc[1]['Feature']} (Engagement/Size)")
print(f"   PC2: {pc2_df.iloc[0]['Feature']} vs {pc2_df.iloc[-1]['Feature']} (Growth patterns)")

## Part 4: Feature Importance from PCA

In [ ]:
# Which original features drive all principal components?
importance = np.sum(np.abs(pca_full.components_), axis=0)

feature_imp = pd.DataFrame(
    {'Feature': features, 'Importance': importance}
).sort_values('Importance', ascending=False)

print("\n" + "=" * 60)
print("FEATURE IMPORTANCE IN PCA")
print("=" * 60)
print(feature_imp.to_string(index=False))

print(f"\n✅ Top features: {', '.join(feature_imp.head(3)['Feature'].tolist())}")
print(f"⚠️ Least important: {feature_imp.iloc[-1]['Feature']} (redundant?)")

## Part 5: PCA for Denoising

In [ ]:
# Reconstruct using fewer components (removes noise)
pca_denoise = PCA(n_components=3)
X_reduced = pca_denoise.fit_transform(X_scaled)
X_reconstructed = pca_denoise.inverse_transform(X_reduced)

# Reconstruction error
recon_error = np.mean(np.sum((X_scaled - X_reconstructed) ** 2, axis=1))

print(f"Reconstruction with 3 components:")
print(f"  Variance retained: {100 * sum(pca_denoise.explained_variance_ratio_):.1f}%")
print(f"  Reconstruction MSE: {recon_error:.4f}")
print(f"  Information loss: {100 * (1 - sum(pca_denoise.explained_variance_ratio_)):.1f}%")

print("\n💡 Denoising Trade-off:")
print("  More components = more information but more noise")
print("  Fewer components = cleaner data but information loss")
print("  Optimal: Find knee in reconstruction error vs components")

## Part 6: Anomaly Detection via Reconstruction Error

In [ ]:
# Customers with high reconstruction error = anomalies
recon_errors = np.sum((X_scaled - X_reconstructed) ** 2, axis=1)
threshold = np.percentile(recon_errors, 95)

anomalies = recon_errors > threshold
n_anomalies = anomalies.sum()

print(f"\nAnomaly Detection Results:")
print(f"  Threshold (95th percentile): {threshold:.2f}")
print(f"  Anomalous customers: {n_anomalies:,} ({100*n_anomalies/len(df):.1f}%)")

# Characterize anomalies
print("\nAnomalous customers (top 10 by error):")
anom_df = df.copy()
anom_df['recon_error'] = recon_errors
top_anomalies = anom_df.nlargest(10, 'recon_error')
print(top_anomalies[['mrr', 'total_usage', 'tenure_days', 'recon_error']].to_string())

print("\n💡 Business Insight:")
print("  These customers don't fit normal patterns")
print("  Could indicate: new customer type, usage anomaly, or data quality issue")

## Hands-On Exercises

### Exercise 1: Scree Plot
Create a plot of explained variance per component. Find the "elbow" where marginal improvement drops.

In [ ]:
# TODO: Plot explained variance per component
# TODO: Find elbow point
# TODO: Decision: How many components to keep?

### Exercise 2: PCA + Clustering Comparison
1. K-Means on original features
2. K-Means on PCA-reduced features (2-3 components)
3. Compare silhouette scores
4. When does PCA help vs hurt?

In [ ]:
# TODO: Compare clustering with/without PCA

### Exercise 3: Autoencoder Alternative
Compare PCA (linear) to autoencoders (non-linear) for dimensionality reduction. Which preserves more relevant structure?

In [ ]:
# TODO: Explore non-linear dimensionality reduction
# from sklearn.manifold import TSNE, UMAP
# Could explore t-SNE or UMAP for comparison

## Assignment: Visualization Dashboard

**Build:**
1. Scree plot (variance explained)
2. Biplot (data points + feature loadings in 2D)
3. Anomaly visualization (high reconstruction error customers)
4. 3D scatter (PC1, PC2, PC3 colored by segment)

**Insight:** What patterns emerge in reduced dimensional space?

## Key Takeaways

✅ Curse of dimensionality concept  
✅ PCA mechanics and interpretation  
✅ Variance explained by components  
✅ Feature importance from loadings  
✅ Visualization in lower dimensions  
✅ Denoising via reconstruction  
✅ Anomaly detection via reconstruction error  

## 🔜 Next Week: Ensemble Methods

Combine multiple models for superior predictions